In [2]:
# !pip install yahoo_fin

ERROR: pyppeteer 1.0.2 has requirement certifi>=2021, but you'll have certifi 2020.6.20 which is incompatible.


  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24585 sha256=b9850bf7a9d7c4c0b8d3714e68ae3009ea346d4cddcda515d613a0a7534e958e
  Stored in directory: c:\users\annaj\appdata\local\pip\cache\wheels\e5\33\1f\68392720485b3ecf125a69e700baaab7624616deedea2fa6e2
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1279 sha256=803500f68ce3757c921c53e937814979e40d428720ccb84f60e01f0e7af4d26d
  Stored in directory: c:\users\annaj\appdata\local\pip\cache\wheels\75\78\21\68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6071 sha256=5a6a06718ee88a02b82c86e43296d3e663ff31d14c3d5f1a678e0c28fd1e80f0
  Stored in directory: c:\users\annaj\appdata\local\pip\cache\wheels\83\63\2f\117884c3b19d46b64d3d61690333aa80c88dc14050e269c546
Successfully built parse bs4 sgmllib3k


In [161]:
# Reference: https://levelup.gitconnected.com/how-to-get-all-stock-symbols-a73925c16a1b
#
#
# *****************************************************************************
#
# Get all stock market ticker symbols, filter the top 5*N by 
# market-cap (high to low), and then filter the top N by beta (low to high).
# Output: top N low-volatility stocks with market-cap & beta 
#         in the file low_volatility_stocks.csv in the same directory.
# Default: N=200, change by editing that line below
#
# *****************************************************************************
#

import pandas as pd
from yahoo_fin import stock_info as si
import pandas_datareader as web
import math
from operator import itemgetter


# N - top N lowest volatile stocks will be exported to low_volatility_stocks.csv
N=200
# M - top M highest market-cap stocks to filter the complete stock list of over 10K stocks.
M=5*N 

# convert the list returned by the APIs into 4 different dataframes with default index
# and ticker as the only column.
df1 = pd.DataFrame( si.tickers_sp500() )
df2 = pd.DataFrame( si.tickers_nasdaq() )
df3 = pd.DataFrame( si.tickers_dow() )
df4 = pd.DataFrame( si.tickers_other() )

# we use a set to join the symbols from different exchanges without DUPLICATES.
sym1 = set( symbol for symbol in df1[0].values.tolist() )
sym2 = set( symbol for symbol in df2[0].values.tolist() )
sym3 = set( symbol for symbol in df3[0].values.tolist() )
sym4 = set( symbol for symbol in df4[0].values.tolist() )
symbols = set.union( sym1, sym2, sym3, sym4 )

# If the fifth letter in a stock symbol is one of the following, 
# it is not a regular common stock and so we take those out.
# W - warrants, R - rights, P - preferred, Q - bankruptcy.
my_list = ['W', 'R', 'P', 'Q']

del_set = set()
sav_set = set()

for symbol in symbols:
    if len( symbol ) > 4 and symbol[-1] in my_list:
        del_set.add( symbol )
    else:
        sav_set.add( symbol )
        
print( f'Removed {len( del_set )} unqualified stock symbols...' )
print( f'There are {len( sav_set )} qualified stock symbols...' )

# create a file with all tickers if needed - one ticker per line.
# with open("all_tickers.txt", "w") as f:
#    f.write( "\n".join(s for s in sav_set) )

# FOR TESTING WITH SUBSET OF STOCK TICKERS.
# tickers = ['AAPL', 'AMZN']
# X=10*M
# tickers = list(sav_set)[0:X]
# print(tickers)

# USE THE ENTIRE LIST of ~10K stock tickers.
tickers = list(sav_set)

market_caps=[]
market_tickers=[]
for ticker in tickers: 
    try:
        mcap = web.get_quote_yahoo(ticker)['marketCap'].get(ticker)
        if math.isnan(mcap):
           continue
        # print(f"{ticker}, {mcap, type(mcap)}")
        market_caps.append(mcap)
        market_tickers.append(ticker)
        # print(market_data)
    except Exception:
        # print("Error.")
        pass
# market_data
df=pd.DataFrame(market_caps, columns=['Market Cap'], index=market_tickers)
df.sort_values('Market Cap', axis=0, ascending=False, inplace=True)

df_top_mktcaps = df.head(M)

market_betas=[]
tickers_to_drop=[]
for ticker, _ in df_top_mktcaps.iterrows(): 
    try:
        # we use  yahoo_fin API for beta value as pandas_datareader
        # does not provide this data value.
        beta = si.get_quote_table(ticker)['Beta (5Y Monthly)']
        if math.isnan(beta):
            tickers_to_drop.append(ticker)
            continue
        # print(f"{ticker}, {beta, math.isnan(beta)}")
        market_betas.append(beta)
    except Exception:
        # print("Error.")
        pass
# market_data
df_temp = df_top_mktcaps.drop(tickers_to_drop, inplace=False)
df_temp["Beta"] = market_betas
# df=pd.DataFrame(zip(market_caps, market_betas), columns=['Market Cap', 'Beta'], index=market_tickers)
df_temp.sort_values('Beta', axis=0, ascending=True, inplace=True)

df_top_low_volatility = df_temp.head(N)

df_top_low_volatility.to_csv("low_volatility_stocks.csv")



Removed 1044 unqualified stock symbols...
There are 10912 qualified stock symbols...


In [74]:
# **** notepad tryouts
from yahoo_fin import stock_info as si

si.get_quote_table('MSFT')

{'1y Target Est': 296.91,
 '52 Week Range': '213.43 - 344.30',
 'Ask': '0.00 x 800',
 'Avg. Volume': 29869036.0,
 'Beta (5Y Monthly)': 0.93,
 'Bid': '245.91 x 1300',
 "Day's Range": '247.34 - 254.20',
 'EPS (TTM)': 9.29,
 'Earnings Date': 'Jan 23, 2023 - Jan 27, 2023',
 'Ex-Dividend Date': 'Feb 15, 2023',
 'Forward Dividend & Yield': '2.72 (1.06%)',
 'Market Cap': '1.856T',
 'Open': 253.72,
 'PE Ratio (TTM)': 26.8,
 'Previous Close': 257.22,
 'Quote Price': 249.00999450683594,
 'Volume': 35568093.0}

In [75]:
# **** notepad tryouts
from yahoo_fin import stock_info as si

list(si.get_quote_table('MSFT'))

['1y Target Est',
 '52 Week Range',
 'Ask',
 'Avg. Volume',
 'Beta (5Y Monthly)',
 'Bid',
 "Day's Range",
 'EPS (TTM)',
 'Earnings Date',
 'Ex-Dividend Date',
 'Forward Dividend & Yield',
 'Market Cap',
 'Open',
 'PE Ratio (TTM)',
 'Previous Close',
 'Quote Price',
 'Volume']

In [78]:
# **** notepad tryouts
list(sav_set)

['MSOS',
 '',
 'RCD',
 'PEV',
 'NAT',
 'EDOC',
 'LEA',
 'TYLG',
 'QSI',
 'FISR',
 'SCHL',
 'RAD',
 'SCC',
 'DRMA',
 'BICK',
 'SSPX',
 'AXLA',
 'LBTYB',
 'KLAQU',
 'KLCD',
 'QCOM',
 'CVLT',
 'THAC',
 'LARK',
 'SARK',
 'CNEY',
 'BIPH',
 'DS$C',
 'FUL',
 'YCBD$A',
 'JSCP',
 'CAG',
 'CHN',
 'FDEM',
 'CHNR',
 'ALTUU',
 'PUBM',
 'NIMC',
 'ASTI',
 'KERN',
 'VICR',
 'QDEF',
 'BIGC',
 'TFII',
 'FIW',
 'DWMF',
 'DILA',
 'LGST',
 'BBD',
 'BRFH',
 'BKYI',
 'ADV',
 'SGOV',
 'RAMMU',
 'OPP$A',
 'SKT',
 'IOACU',
 'FKWL',
 'SNPO',
 'HPE',
 'MFIC',
 'BTAL',
 'CENX',
 'CF',
 'NTNX',
 'BCOV',
 'TXG',
 'SFLR',
 'BCO',
 'ACQRU',
 'SPEM',
 'ADEX.U',
 'BR',
 'GRCY',
 'AIVI',
 'XPRO',
 'EMBK',
 'VRAY',
 'BOTJ',
 'SNN',
 'RBA',
 'FAD',
 'NLR',
 'GTEK',
 'NLS',
 'IVW',
 'JD',
 'PFE',
 'SCOR',
 'MA',
 'KWE',
 'SWIM',
 'MAT',
 'SONX',
 'VOOV',
 'CNNB',
 'ELBM',
 'FFIN',
 'MYNZ',
 'ECC',
 'PP',
 'NTRS',
 'TSVT',
 'ZVZZT',
 'MDU',
 'PARAA',
 'GBCI',
 'XTN',
 'QLVD',
 'ZUO',
 'ESLT',
 'EAD',
 'TSAT',
 'JHSC',
 'CBX'

In [92]:
# **** notepad tryouts
from operator import itemgetter

list(itemgetter('Market Cap', 'Beta (5Y Monthly)')(si.get_quote_table('MSFT')))


['1.856T', 0.93]

In [91]:
# **** notepad tryouts
list(sav_set)[0:5]

['MSOS', '', 'RCD', 'PEV', 'NAT']

In [110]:
# **** notepad tryouts
l = list((2, 3))
l.insert(0, 1)
print(l)

[1, 2, 3]


In [124]:
# **** notepad tryouts
import pandas_datareader as web

web.get_quote_yahoo('MSFT')['marketCap'].get('MSFT')

1829699715072

In [145]:
# **** notepad tryouts
import math
from yahoo_fin import stock_info as si

beta = si.get_quote_table('PEV')['Beta (5Y Monthly)']
math.isnan(beta)

True

In [122]:
# **** notepad tryouts
import pandas_datareader as web

list(web.get_quote_yahoo('MSFT')['marketCap'])

[1856237600768]

In [45]:
# **** notepad tryouts
import pandas_datareader as web

list(web.get_quote_yahoo(ticker))

['language',
 'region',
 'quoteType',
 'typeDisp',
 'quoteSourceName',
 'triggerable',
 'customPriceAlertConfidence',
 'currency',
 'exchange',
 'shortName',
 'longName',
 'messageBoardId',
 'exchangeTimezoneName',
 'exchangeTimezoneShortName',
 'gmtOffSetMilliseconds',
 'market',
 'esgPopulated',
 'regularMarketChangePercent',
 'regularMarketPrice',
 'marketState',
 'financialCurrency',
 'regularMarketOpen',
 'firstTradeDateMilliseconds',
 'priceHint',
 'postMarketChangePercent',
 'postMarketTime',
 'postMarketPrice',
 'postMarketChange',
 'regularMarketChange',
 'regularMarketTime',
 'regularMarketDayHigh',
 'regularMarketDayRange',
 'regularMarketDayLow',
 'regularMarketVolume',
 'regularMarketPreviousClose',
 'bid',
 'ask',
 'bidSize',
 'askSize',
 'fullExchangeName',
 'averageDailyVolume3Month',
 'averageDailyVolume10Day',
 'fiftyTwoWeekLowChange',
 'fiftyTwoWeekLowChangePercent',
 'fiftyTwoWeekRange',
 'fiftyTwoWeekHighChange',
 'fiftyTwoWeekHighChangePercent',
 'fiftyTwoWeekLow'

In [47]:
# **** notepad tryouts
from yahoo_fin import stock_info as si

list(si.get_quote_table(ticker))

['1y Target Est',
 '52 Week Range',
 'Ask',
 'Avg. Volume',
 'Beta (5Y Monthly)',
 'Bid',
 "Day's Range",
 'EPS (TTM)',
 'Earnings Date',
 'Ex-Dividend Date',
 'Forward Dividend & Yield',
 'Market Cap',
 'Open',
 'PE Ratio (TTM)',
 'Previous Close',
 'Quote Price',
 'Volume']

In [70]:
# **** notepad tryouts
from yahoo_fin import stock_info as si

si.tickers_sp500()

['A',
 'AAL',
 'AAP',
 'AAPL',
 'ABBV',
 'ABC',
 'ABMD',
 'ABT',
 'ACGL',
 'ACN',
 'ADBE',
 'ADI',
 'ADM',
 'ADP',
 'ADSK',
 'AEE',
 'AEP',
 'AES',
 'AFL',
 'AIG',
 'AIZ',
 'AJG',
 'AKAM',
 'ALB',
 'ALGN',
 'ALK',
 'ALL',
 'ALLE',
 'AMAT',
 'AMCR',
 'AMD',
 'AME',
 'AMGN',
 'AMP',
 'AMT',
 'AMZN',
 'ANET',
 'ANSS',
 'AON',
 'AOS',
 'APA',
 'APD',
 'APH',
 'APTV',
 'ARE',
 'ATO',
 'ATVI',
 'AVB',
 'AVGO',
 'AVY',
 'AWK',
 'AXP',
 'AZO',
 'BA',
 'BAC',
 'BALL',
 'BAX',
 'BBWI',
 'BBY',
 'BDX',
 'BEN',
 'BF-B',
 'BIIB',
 'BIO',
 'BK',
 'BKNG',
 'BKR',
 'BLK',
 'BMY',
 'BR',
 'BRK-B',
 'BRO',
 'BSX',
 'BWA',
 'BXP',
 'C',
 'CAG',
 'CAH',
 'CARR',
 'CAT',
 'CB',
 'CBOE',
 'CBRE',
 'CCI',
 'CCL',
 'CDAY',
 'CDNS',
 'CDW',
 'CE',
 'CEG',
 'CF',
 'CFG',
 'CHD',
 'CHRW',
 'CHTR',
 'CI',
 'CINF',
 'CL',
 'CLX',
 'CMA',
 'CMCSA',
 'CME',
 'CMG',
 'CMI',
 'CMS',
 'CNC',
 'CNP',
 'COF',
 'COO',
 'COP',
 'COST',
 'CPB',
 'CPRT',
 'CPT',
 'CRL',
 'CRM',
 'CSCO',
 'CSGP',
 'CSX',
 'CTAS',
 'CTLT',
 'C

In [71]:
# **** notepad tryouts
import pandas_datareader as web

web.get_quote_yahoo('AAPL')['marketCap']

AAPL    2171455668224
Name: marketCap, dtype: int64

In [72]:
# **** notepad tryouts
from yahoo_fin import stock_info as si

si.get_quote_table('AAPL')['Beta (5Y Monthly)']

1.22